# COMP0189: Applied Artificial Intelligence
## Week 9 (Clustering)


### 🎯 Objectives
1. To learn how to apply different clustering approaches (K-Means and Gaussian Mixture Models) to different tasks: clustering images and clustering voxels (image segmentation)
2. To learn how to quantify clustering results



### Acknowledgements
- Many thanks to Prof. John Ashburner for kindly providing the brain imaging data.
- https://scikit-learn.org/stable/
- https://en.wikipedia.org/wiki/MNIST_database
- https://brain-development.org/ixi-dataset/



In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# Part 1: MNIST dataset

In this part we will apply Principal Component Analysis (PCA) to the MNIST dataset and use K-Means to cluster the digits.

https://scikit-learn.org/stable/modules/clustering.html#k-means

### Task 1: Load MNIST data and assemble it in two matrices X (images) and y (labels)

In [ ]:
MNIST = np.load("mnist.npz")
for k in MNIST.files:
    print(k)

In [ ]:
MNIST["X"].shape, MNIST["y"].shape

In [ ]:
mnist_X = MNIST[None]
mnist_y = MNIST[None]

### Task 2: Visualise the data for better understanding

In [ ]:
def make_img_grid(images, n_cols=10):
    """Helper function for arranging images into a grid"""
    cols = []
    gap = len(images) % n_cols
    if gap > 0:
        # add padding if needed
        images = np.concatenate(
            (images, np.zeros((n_cols - gap,) + images[0].shape)), 0
        )
    for n in range(n_cols):
        cols.append(np.concatenate(images[np.arange(n, len(images), step=n_cols)]))
    return np.concatenate(cols, -1)


plt.figure(figsize=(14, 7))
plt.imshow(make_img_grid(mnist_X[:None], n_cols=30), cmap="binary_r")

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(12, 12))
plt.gray()

# loop through subplots and add mnist images
for i, ax in enumerate(axs.flat):
    ax.imshow(mnist_X[None])
    ax.axis("off")
    ax.set_title("Number {}".format(mnist_y[i]))

# display the figure
plt.show()

In order to apply PCA to the MNIST data we need to reshape the original MNIST data, from

    mnist_images.shape == [60000, 28, 28]

into a 2d array (matrix)

    X_mnist.shape == [60000, 784]


In [ ]:
# Reshaping
mnist_X_original = mnist_X.copy()
mnist_X = mnist_X.reshape(len(mnist_X), -1)
print(mnist_X.shape)

### Task 3: Apply PCA to the MNIST data

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
mnist_X_pca = pca.None

In [ ]:
mnist_X_pca.shape

### Task 4: Plot the MNIST data projected onto the first two principal components (using different colours for the different digits). Use the labels to colour the examples

In [ ]:
plot = plt.scatter(mnist_X_pca[:, None], mnist_X_pca[:, None], c=mnist_y, cmap="rainbow")
plt.legend(handles=plot.legend_elements()[0], labels=[x for x in range(10)])
plt.show()

### Task 5: Plot the explained variance per component

In [ ]:
# Determine explained variance using explained_variance_ratio_ attribute
# explained_variance_ratio_
#   Percentage of variance explained by each of the selected components.
#   If n_components is not set then all components are stored and the sum of the ratios is equal to 1.0.
exp_var_pca = pca.None  # shape: (784,)

# Cumulative sum of eigenvalues will be used to create step plot
# for visualizing the variance explained by each principal component.
cum_sum_eigenvalues = np.cumsum(exp_var_pca)

# Create the plot
plt.bar(
    range(0, len(exp_var_pca)),
    exp_var_pca,
    alpha=0.5,
    align="center",
    label="Individual explained variance",
)
plt.step(
    range(0, len(cum_sum_eigenvalues)),
    cum_sum_eigenvalues,
    where="mid",
    label="Cumulative explained variance",
)
plt.ylabel("Explained variance ratio")
plt.xlabel("Principal component index")
plt.legend(loc="best")
plt.tight_layout()
plt.show()

### Manual way of gettting explained variance

In [ ]:
def compute_PCA_parameters(X, M):
    """
    This function computes the first M prinicpal components of a
    dataset X. It returns the mean of the data, the projection matrix,
    and the associated singular values.

    While you can compute this however you want, `np.linalg.svd` is
    highly recommended. Please look at its documentation to choose
    its arguments appropriately, and on how to interpret its return values.

    INPUT:
    X    : (N, D) matrix; each row is a D-dimensional data point
    M    : integer, <= D (number of principal components to return)

    OUTPUTS:
    x_bar  : (D,) vector, with the mean of the data
    W      : (D, M) semi-orthogonal matrix of projections
    s      : (D,) vector of singular values
    """
    N, D = X.shape
    x_bar = np.mean(X, 0)
    X_bar = X - x_bar
    u, s, vh = np.linalg.svd(X_bar, full_matrices=False)
    W = vh.T[:, :M]
    return x_bar, W, s

In [ ]:
mnist_mean, W_mnist, s_mnist = compute_PCA_parameters(mnist_X, 50)

N, data_dim = mnist_X.shape
plt.figure(figsize=(12, 4))
plt.plot(np.arange(data_dim) + 1, s_mnist**2 / N, ".-")
plt.xlabel("Component")
plt.ylabel("Explained variance")

### Task 5: Apply KMeans to cluster the MNIST data

- Preprocess the MNIST dataset with PCA to compress it down to a 2-dimensional feature space before applying the K-Means

- Try K-Means with different numbers of clusters and use the Silhouette Coefficient to choose the optimal number of cluster

Discussion: Does the Silhouette Coefficient chooses the right number of clusters?

In [ ]:
from sklearn.cluster import KMeans
pca = PCA(n_components=None)
mnist_X_pca = pca.None
n_digits = len(np.unique(mnist_y))
print(n_digits)

In [ ]:
%%time

kmeans = KMeans(n_clusters=None, n_init=10)
kmeans.fit(None)

In [ ]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples, silhouette_score

# Generating the sample data from make_blobs
# This particular setting has one distinct cluster and 3 clusters placed close
# together.
X = None
y = None
range_n_clusters = [2, 5,10]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters,  n_init=10 , random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(
        X[:, 0], X[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
    )

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(
        centers[:, 0],
        centers[:, 1],
        marker="o",
        c="white",
        alpha=1,
        s=200,
        edgecolor="k",
    )

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d"
        % n_clusters,
        fontsize=14,
        fontweight="bold",
    )

plt.show()

### Task 6: Apply the K-Means to the MNIST dataset compressed to 2, 10 and 100 PCs (fixing the number of clusters to 10) and quantify the performance of the three cluster models using different clustering metrics

See "Quantifying the quality of clustering results" in https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#sphx-glr-auto-examples-text-plot-document-clustering-py

In [ ]:
from collections import defaultdict
from time import time

from sklearn import metrics

evaluations = []
evaluations_std = []
labels = mnist_y

def fit_and_evaluate(km, X, name=None, n_runs=5):
    name = km.__class__.__name__ if name is None else name

    train_times = []
    scores = defaultdict(list)
    for seed in range(n_runs):
        km.set_params(random_state=seed)
        t0 = time()
        km.fit(X)
        train_times.append(time() - t0)
        scores["Homogeneity"].append(metrics.homogeneity_score(labels, km.labels_))
        scores["Completeness"].append(metrics.completeness_score(labels, km.labels_))
        scores["V-measure"].append(metrics.v_measure_score(labels, km.labels_))
        scores["Adjusted Rand-Index"].append(
            metrics.adjusted_rand_score(labels, km.labels_)
        )
        scores["Silhouette Coefficient"].append(
            metrics.silhouette_score(X, km.labels_, sample_size=2000)
        )
    train_times = np.asarray(train_times)

    print(f"clustering done in {train_times.mean():.2f} ± {train_times.std():.2f} s ")
    evaluation = {
        "estimator": name,
        "train_time": train_times.mean(),
    }
    evaluation_std = {
        "estimator": name,
        "train_time": train_times.std(),
    }
    for score_name, score_values in scores.items():
        mean_score, std_score = np.mean(score_values), np.std(score_values)
        print(f"{score_name}: {mean_score:.3f} ± {std_score:.3f}")
        evaluation[score_name] = mean_score
        evaluation_std[score_name] = std_score
    evaluations.append(evaluation)
    evaluations_std.append(evaluation_std)

In [ ]:
pca = PCA(n_components=None)
mnist_X_pca = pca.None
kmeans = KMeans(n_clusters=n_digits, n_init=10)
fit_and_evaluate(
    None,
    None,
    name="2-PCA-Kmeans",
)
pca = PCA(n_components=10)
mnist_X_pca = pca.None
kmeans = KMeans(n_clusters=n_digits, n_init=10)
fit_and_evaluate(
    None,
    None,
    name="10-PCA-Kmeans",
)

pca = PCA(n_components=100)
mnist_X_pca = pca.None
kmeans = KMeans(n_clusters=n_digits, n_init=10)
fit_and_evaluate(
    None,
    None,
    name="100-PCA-Kmeans",
)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(16, 6), sharey=True)

df = pd.DataFrame(evaluations[::-1]).set_index("estimator")
df_std = pd.DataFrame(evaluations_std[::-1]).set_index("estimator")

df.drop(
    ["train_time"],
    axis="columns",
).plot.barh(ax=ax0, xerr=df_std)
ax0.set_xlabel("Clustering scores")
ax0.set_ylabel("")

df["train_time"].plot.barh(ax=ax1, xerr=df_std["train_time"])
ax1.set_xlabel("Clustering time (s)")
plt.tight_layout()

# Part 2: IXI dataset

In this part we apply use Gaussian Mixture Model (GMM) to an image segmentation task. We will use the voxel intensities of two brain images to cluster them into four different clusters that should correspond to different types of brain tissue (grey matter, white amtter, Cerebrospinal fluid (CSF) and "other").

https://scikit-learn.org/stable/modules/mixture.html#gmm

In [ ]:
!pip install nilearn

### Task 8: Load two brain images (proton density-weighted (PDW) and T2-weighted (T2W)) that have been previously pre-processed

In [24]:
import nibabel
f = nibabel.load("/content/T2.nii")
x1 = f.get_fdata()

f = nibabel.load("/content/PD.nii")
x2 = f.get_fdata()

### Task 9: Vectorize the brain images and combine them to create a data matrix with 2 dimensions per voxel (PD and T2 intensities)

In [25]:
# Flatten the 3D arrays to 2D arrays where each row is a voxel
x1_flat = x1.reshape(None, 1) # Reshape x1 to have one column and as many rows as there are voxels
x2_flat = x2.reshape(None, 1) # Reshape x2 similarly

# Combine the flattened arrays side by side to create a new 2D array with two columns (PD and T2 intensities)
data_matrix = np.hstack((None))

In [ ]:
print(data_matrix.shape)

In [27]:
# reshaping to original
original_shape = x1.shape

# Split the 2D data_matrix back into two 1D arrays
x_1_flat = data_matrix[:, 0]  # This extracts the first column (PD intensities)
x_2_flat = data_matrix[:, 1]  # This extracts the second column (T2 intensities)

# Reshape these 1D arrays back into their original 3D shape
x1_reshaped = x_1_flat.reshape(None)
x2_reshaped = x_2_flat.reshape(None)


In [ ]:
print(x_1_flat.shape)

In [ ]:
# check if original and un-reshape reshaped match
print(np.array_equal(x1, x1_reshaped))
print(np.array_equal(x2, x2_reshaped))


###  Task 10: Apply GMM to cluster the voxels fixing the number of cluster to 4

In [30]:
from sklearn.mixture import GaussianMixture

# Assuming data_matrix is the combined 2D array from the previous steps

# Initialize the Gaussian Mixture Model with 4 components (clusters)
gmm = GaussianMixture(n_components=None, random_state=0)

# Fit the model to the data and predict the cluster for each voxel
cluster_labels = gmm.fit_predict(None)

# cluster_labels is now a 1D array with the cluster label (0, 1, 2, 3) for each voxel


In [ ]:
print(cluster_labels.shape)

In [ ]:
# Reshape cluster_labels back to the original 3D shape of the images
clustered_image = cluster_labels.reshape(None)

# clustered_image is a 3D array with the same shape as the original images,
# where each voxel's value represents its cluster label.
print(clustered_image.shape)

In [ ]:
import nibabel
from nilearn import plotting

# Convert the clustered image to a Nifti1Image object using the correct affine matrix
clustered_img_nii = nibabel.Nifti1Image(clustered_image.astype(np.int16), affine=f.affine)

# View the clustered image in an interactive viewer
plotting.view_img(None, threshold='auto', cmap='nipy_spectral')


Alternative, instead of classes, based on probabilities

In [34]:
# Predict the probabilities for each voxel belonging to each cluster
voxel_probabilities = gmm.predict_proba(None)

# Now, voxel_probabilities is a 2D numpy array where each row represents a voxel
# and each column represents the probability of that voxel belonging to a certain cluster.

# Reshape these probabilities back to the original 3D shape for each cluster
probabilities_4d = np.zeros((x1.shape[0], x1.shape[1], x1.shape[2], gmm.n_components))

for i in range(gmm.n_components):
    # Reshape the probabilities for cluster i into the original 3D shape
    probabilities_4d[:,:,:,i] = voxel_probabilities[:,i].reshape(x1.shape)


In [ ]:
from nilearn import plotting

# Convert the probability maps to NIfTI images and visualize
prob_img_nii = nibabel.Nifti1Image(probabilities_4d[:,:,:,None].astype(np.float32), affine=f.affine)
plotting.view_img(prob_img_nii, threshold='auto', cmap='hot', title=f'Cluster {0} Probability Map')



In [ ]:
prob_img_nii = nibabel.Nifti1Image(probabilities_4d[:,:,:,None].astype(np.float32), affine=f.affine)
plotting.view_img(prob_img_nii, threshold='auto', cmap='hot', title=f'Cluster {1} Probability Map')

In [ ]:
prob_img_nii = nibabel.Nifti1Image(probabilities_4d[:,:,:,None].astype(np.float32), affine=f.affine)
plotting.view_img(prob_img_nii, threshold='auto', cmap='hot', title=f'Cluster {2} Probability Map')

In [ ]:
prob_img_nii = nibabel.Nifti1Image(probabilities_4d[:,:,:,None].astype(np.float32), affine=f.affine)
plotting.view_img(prob_img_nii, threshold='auto', cmap='hot', title=f'Cluster {2} Probability Map')

#### As above but now ignoring the background class and using 3 clusters

In [ ]:
import numpy as np
import nibabel
from sklearn.mixture import GaussianMixture
from nilearn import plotting

# Load the images
f_t2 = nibabel.load("/content/T2.nii")
x1 = f_t2.get_fdata()
f_pd = nibabel.load("/content/PD.nii")
x2 = f_pd.get_fdata()

# Flatten the 3D arrays to 2D arrays where each row is a voxel
x1_flat = x1.reshape(-1, 1) # Reshape x1 to have one column and as many rows as there are voxels
x2_flat = x2.reshape(-1, 1) # Reshape x2 similarly

# Combine the flattened arrays side by side
data_matrix = np.hstack((None))

# Mask for non-zero pixels
non_zero_mask = np.any(data_matrix != 0, axis=1)

# Filter out the zero-valued pixels
data_matrix_non_zero = data_matrix[None]

# Initialize the Gaussian Mixture Model
gmm = GaussianMixture(n_components=None, random_state=0)

cluster_labels_non_zero = gmm.fit_predict(None)

# Adjust the labels to start from 1 instead of 0 (for visualisaiton purposes)
cluster_labels_non_zero += 1

# Initialize an output array with zeros for the background
clustered_image = np.zeros(x1_flat.shape[0], dtype=np.int16)

# Assign the GMM cluster labels back to the non-zero pixels in the output array
clustered_image[non_zero_mask] = cluster_labels_non_zero

# Reshape cluster_labels back to the original 3D shape of the images
clustered_image_reshaped = clustered_image.reshape(None)

# Convert the clustered image to a Nifti1Image object using the correct affine matrix
clustered_img_nii = nibabel.Nifti1Image(clustered_image_reshaped, affine=f_t2.affine)

# View the clustered image
plotting.view_img(None, threshold='auto', cmap='nipy_spectral')


In [40]:
# Predict the probabilities for each voxel belonging to each cluster
voxel_probabilities = gmm.predict_proba(data_matrix_non_zero)

# Initialize an output 4D array for probabilities with zeros
# This array has the same width, height, and depth as the original images, and an extra dimension for each cluster
probabilities_4d = np.zeros((x1.shape[0], x1.shape[1], x1.shape[2], gmm.n_components))

# We need a way to map the probabilities back to their original positions including zeros
# Flatten the 4D array to match the shape of the non-zero processing (x1_flat.shape[0], n_components)
probabilities_flat = probabilities_4d.reshape(None, gmm.n_components)

# Assign the probabilities back to the non-zero positions in the flattened array
probabilities_flat[non_zero_mask, :] = voxel_probabilities

# Reshape this flat probabilities array back to the original 4D shape
probabilities_4d = probabilities_flat.reshape(x1.shape[0], x1.shape[1], x1.shape[2], gmm.n_components)




In [ ]:
from nilearn import plotting

# Convert the probability maps to NIfTI images and visualize
prob_img_nii = nibabel.Nifti1Image(probabilities_4d[:,:,:,None].astype(np.float32), affine=f.affine)
plotting.view_img(prob_img_nii, threshold='auto', cmap='hot', title=f'Cluster {0} Probability Map')



In [ ]:
prob_img_nii = nibabel.Nifti1Image(probabilities_4d[:,:,:,None].astype(np.float32), affine=f.affine)
plotting.view_img(prob_img_nii, threshold='auto', cmap='hot', title=f'Cluster {1} Probability Map')

In [ ]:
prob_img_nii = nibabel.Nifti1Image(probabilities_4d[:,:,:,None].astype(np.float32), affine=f.affine)
plotting.view_img(prob_img_nii, threshold='auto', cmap='hot', title=f'Cluster {2} Probability Map')